# 데이터 전처리

In [31]:
# 크롤러를 만들기 전 필요한 도구들을 임포트합니다.
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 페이지 수, 카테고리, 날짜를 입력값으로 받습니다.
def make_urllist(page_num, code, date): 
  urllist= []
  for i in range(1, page_num + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)   
    news = requests.get(url)

    # BeautifulSoup의 인스턴스 생성합니다. 파서는 html.parser를 사용합니다.
    soup = BeautifulSoup(news.content, 'html.parser')

    # CASE 1
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    # CASE 2
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
    # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져옵니다.
    for line in news_list:
        urllist.append(line.a.get('href'))
  return urllist

In [33]:
url_list = make_urllist(2, 101, 20200506)
print('뉴스 기사의 개수: ',len(url_list))

뉴스 기사의 개수:  40


In [34]:
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

In [35]:
from newspaper import Article

#- 데이터프레임을 생성하는 함수입니다.
def make_data(urllist, code):
  text_list = []
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)

  #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류로 붙여줍니다.
  df = pd.DataFrame({'news': text_list})

  #- 데이터프레임의 'code' 키 아래 한글 카테고리명을 붙여줍니다.
  df['code'] = idx2word[str(code)]
  return df

In [37]:
data = make_data(url_list, 101)
#- 상위 10개만 출력해봅니다.
data[:10]

,news,code
0,고려은단이 5월을 맞아 응원 메시지를 공유하는 ‘5월 5글자로 응원 부탁해!’ 이벤...,경제
1,코리아나화장품의 민감성 피부를 위한 저자극 스킨케어 브랜드 '프리엔제'가 마르고 건...,경제
2,서울장수주식회사가 부드럽고 달콤한 맛으로 인기를 모으고 있는 생막걸리 ‘인생막걸리’...,경제
3,[서울=뉴시스] 오동현 기자 = 모바일 게임 기업 컴투스는 3D 모바일 야구 게임 ...,경제
4,대원제약이 2020년 상반기 신입과 경력 정기 공채를 실시합니다.정기 공채 모집분야...,경제
5,"[AFP=연합뉴스] [AFP=연합뉴스]\n\n""요즘은 잔인한 날""…리프트도 앞서 9...",경제
6,이재용 삼성전자 부회장이 6일 삼성전자 서울 서초사옥에서 대국민 사과 회견을 하기 ...,경제
7,JW중외제약이 A형 혈우병 예방요법제 ‘헴리브라피하주사를 출시하고 본격적인 마케팅 ...,경제
8,"옵티팜과 휴벳바이오가 공동 개발중인 백신 후보 물질에 대해 마우스, 기니피그, 미니...",경제
9,[한국경제TV 신동호 기자]\n\n전남 나주시와 충북 청주시가 방사광 가속기 구축사...,경제


In [27]:
code_list = [102, 103, 105]

code_list

[102, 103, 105]

In [38]:
def make_total_data(page_num, code_list, date):
  df = None

  for code in code_list:
    url_list = make_urllist(page_num, code, date)
    df_temp = make_data(url_list, code)
    print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

    if df is not None:
      df = pd.concat([df, df_temp])
    else:
      df = df_temp

  return df

In [39]:
df = make_total_data(1, code_list, 20200506)

102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.


In [43]:
df.sample(10)

,news,code
2,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,IT/과학
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
13,블랙홀을 품은 삼중성계 HR 6819 상상도 [ESO/L. Calcada 제공/ 재...,IT/과학
6,기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...,생활/문화
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
5,나주 방사광 가속기 조감도. 전남도 제공 나주 방사광 가속기 조감도. 전남도 제공\...,IT/과학
7,[한국경제TV 김주리 기자]\n\n지구에서 약 1천광년밖에 떨어지지 않은 곳에서 맨...,IT/과학
15,©나스(NARS) ©나스(NARS)\n\n모던 메이크업 아티스트 브랜드 나스(NAR...,생활/문화
18,기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...,사회
15,/뉴스1 DB. /뉴스1 DB.\n\n(서울=뉴스1) 이상학 기자 = 밤에 귀가하던...,사회


### CSV파일 불러오기

In [69]:
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data.csv"
df = pd.read_table(csv_path, sep=',')
df.head()

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회


In [77]:
# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거합니다.
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
4825    신종 코로나바이러스 감염증코로나 사태 이후 가정의 달 월에도 언택트비대면 신풍속도가...
4826    는 소비자로부터 월 이용료 만만원을 받고 초고속 인터넷을 제공한다 그런 브로드밴드가...
4827    머리를 긁고 있는 오랑우탄 몸을 긁는 행동을 따라 하는 것은 부정적 감정과 관련이 ...
4828    가 오는 일 정식 출시하는 스마트폰 벨벳이 사실상 공짜폰이 될 전망이다 단말기 가격...
4829    이미지제공게티이미지뱅크 이미지제공게티이미지뱅크  전자신문  전자신문인터넷 무단전재 ...
Name: news, Length: 3996, dtype: object

In [70]:
print(df.isnull().sum())   #Null 값 여부 확인

news    0
code    0
dtype: int64


In [71]:
# 중복된 샘플들을 제거합니다.
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  3996


In [72]:
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  3996


# 1. 형태소 토큰화 및 비교

## Mecab 

In [78]:
from konlpy.tag import Mecab
tokenizer = Mecab()


# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data


In [79]:
import time
start = time.time()  # 시작 시간 저장

text_data = preprocessing(df['news'])
print(text_data[0])
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 인 이상 가구 기준 으로 만 원 받 게 되 며 인 가구 만 원 인 가구 만 원 인 가구 만 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 원 지급 고 어 시민 국비 지원금 만 지급 며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만 원 경기도 재난 기본소득 만 원 정부 긴급 재난 지원금 만 원 총 만 원 받 게 된다 취약 계층 아닌 시민 오 월일 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 시 일 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태 스 크 포스 를 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자 재 배포
time : 5.248368501663208


## Hannanum

In [87]:
from konlpy.tag import Hannanum
hannanum = Hannanum()

def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = hannanum.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data



In [81]:
import time
start = time.time()  # 시작 시간 저장

text_data = preprocessing(df['news'])
print(text_data[0])
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기 자 파 아 주 시는 일 관내 취약계층 만가구 대 어 어 정부 긴급재난지원금 입금 완료했다파주시민 받 긴급재난지원금 일 ㄴ 이상 가구 기준 으로 만원 받 게 되 며 일 ㄴ 가구 만원 일 ㄴ 가구 만원 일 ㄴ 가구 만원이다정부 발표 ㄴ 긴급재난지원금 파주시민 지급 ㄴ 금액 다르 ㄴ 이유 국비지방비 부담 비율 때문이다파주시 이미 모든 시민 경기도파주시 재난기본소득인당 각 만원 지급 하고 어 시민 국비 지원금 만 지급 며 일 ㄴ 가구 기준 으로 총 지원 금액 파주시 재난기본소득 만원 경기 도 재난기본소득 만원 정부 긴급재난지원금 만원 총 만원 받 게 된다취약계층 아니 ㄴ 시민 오 월일 부터 소 지 하고 신용체크카드사 홈페이지 에서 긴급재난지원금 지원 신청 ㄹ 세대주 가족 지원금 일괄 신청 어야 한다한편 파 아 주 시는 일 김정기 부시장 단장 으로 긴급재난지원금 추진 태스크포스 를 구성해 긴급재난지원금 원활 게 지급 되 ㄹ 도록 지원 ㄴ다 저작권자 파이낸셜뉴스 전재재배포
time : 150.68765592575073


## Kkma

In [88]:
from konlpy.tag import Kkma
kkma = Kkma()

def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = kkma.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data


In [89]:
import time
start = time.time()  # 시작 시간 저장

text_data = preprocessing(df['news'])
print(text_data[0])
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

파주 시청 사진 제공 파주시 파주 시청 사진 제공 파주시 파주 강 근주 파주시 일 관내 취약 계층 만 가구 대하 어 정부 긴급 재난 지원금 입금 완료 었 파주 시민 받 긴급 재난 지원금 인 이상 가구 기준 으로 만원 받 게 되 며 ㄴ 가구 만원 인 가구 만원 인 가구 만원 정부 발표 ㄴ 긴급 재난 지원금 파주시 민 지급 ㄴ 금액 다르 ㄴ 이유 국비 지방비 부담 비율 파주시 이미 모든 시민 경기도 파주시 재난 기본 소득 ㄴ 당 각 만원 지급 고 어 시민 국비 지원금 만 지급 며 ㄴ 가구 기준 으로 총 지원 금액 파주시 재난 기본 소득 만원 경기도 재난 기본 소득 만원 정부 긴급 재난 지원금 만원 총 만원 받 게 되 ㄴ다 취약 계층 아니 ㄴ 시민 오 월일 부터 소지 고 신용 체크 카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 ㄹ 세대주 가족 지원금 일괄 신청 어야 ㄴ다 한편 파주시 일 김 정기 부시장 단장 으로 긴급 재난 지원금 추진 태스크 포스 를 구성 어 긴급 재난 지원금 원활 게 지급 되 ㄹ 도록 지원 ㄴ다 저작권자 재 배포
time : 1073.7072577476501


## Komoran 

In [82]:
from konlpy.tag import Komoran
komoran = Komoran()

def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = komoran.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data


In [83]:
import time
start = time.time()  # 시작 시간 저장

text_data = preprocessing(df['news'])
print(text_data[0])
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

파주 시청 사진 제공 파주시 파주 시청 사진 제공 파주시 파주 파이낸셜뉴스 강 근 주 파주시 일 관내 취약 계층 만 가구 대하 아 정부 긴급 재난 지원금 입금 완료 았 파주시 민이 받 긴급 재난 지원금 인 이상 가구 기준 으로 만원 받 게 되 며 인 가구 만원 인 가구 만원 인 가구 만원 이다 정부 발표 ㄴ 긴급 재난 지원금 파주시 민 지급 ㄴ 금액 다른 이유 국비 지방비 부담 비율 이다 파주시 이미 모든 시민 경기도 파주시 재난 기본소득 인당 각 만원 지급 고 어 시민 국비 지원금 만 지급 며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만원 경기도 재난 기본소득 만원 정부 긴급 재난 지원금 만원 총 만원 받 게 되 ㄴ 취약 계층 아니 ㄴ 시민 오 월 일 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 ㄹ 세대주 가족 지원금 일괄 신청 아야 한편 파주시 일 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태스크포스를 구성 아 긴급 재난 지원금 원활 게 지급 되 ㄹ 도록 지원 ㄴ다 저작권 자 파이낸셜뉴스 재 배포
time : 61.265748739242554


## Okt

In [84]:
from konlpy.tag import Okt
okt = Okt()

def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = okt.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [85]:
import time
start = time.time()  # 시작 시간 저장

text_data = preprocessing(df['news'])
print(text_data[0])
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

파주 시청 사진 제공 파주시 파주 시청 사진 제공 파주시 파주 파이낸셜뉴스 강 근 주 파주시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지 원금 입금 완료 했다 파주시민 받는 긴급 재난 지원 금은 인 이상 가구 기준 으로 만원 받게 되며 인 가구 만원 인 가구 만원 인 가구 만원 이다 정부 발표 긴급 재난 지 원금 파주시민 지급 금액 다른 이유 국비 지 방비 부담 비율 이다 파주시 이미 모든 시민 경기도 파주시 재난 기본소득 인 당 각 만원 지급 하고 있어 시민 국비 지 원금 만 지급 하며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만원 경기도 재난 기본소득 만원 정부 긴급 재난 지 원금 만원 총 만원 받게 된다 취약 계층 아닌 시민 오는 월일 부터 소지 하고 있는 신용 체크카드 사 홈페이지 에서 긴급 재난 지 원금 지원 신청 있다 세대주 가족 지 원금 일괄 신청 해야 한다 한편 파주시 일 김정기 부시장 단장 으로 하는 긴급 재난 지 원금 추진 태스크포스 를 구성 해 긴급 재난 지 원금 원활하게 지급 될 있도록 지원 저작권 자 파이낸셜뉴스 전 재재 배포
time : 131.06016540527344


### 형태소 비교 분석

1. 연산속도
    Mecab의 연산속도가 압도적으로 빠름   
    Kkma 는 너무 느려 비효율적   
       
2. 성능
    전체적인 성능도 Mecab 이 좋음   
    Okt의 형태소 분류 성능도 좋았지만 Mecab에 비해서 처리속도가 현저히 느림
       
* 결론 : Mecab 
    처리속도는 독보적으로 빠르다   
    성능면에서도 상위권      

# 2. 불용어 제거

* 불용어를 추가해 돌려봤지만 큰변화는 없었다.    
* 너무 많은 불용어를 추가하면 오히려 연산속도가 늘었다. 

In [97]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스',]

In [136]:
stopwords.extend(['파주','채', '돼', '시장', '와', '이상', '원', '게', '되며'
                 '으로', '해', '도록', '에서', '씨', '니다', '카카오톡', '적인'
                  '를','항상', '고'])

In [137]:
stopwords

['에',
 '는',
 '은',
 '을',
 '했',
 '에게',
 '있',
 '이',
 '의',
 '하',
 '한',
 '다',
 '과',
 '때문',
 '할',
 '수',
 '무단',
 '따른',
 '및',
 '금지',
 '전재',
 '경향신문',
 '기자',
 '는데',
 '가',
 '등',
 '들',
 '파이낸셜',
 '저작',
 '등',
 '뉴스',
 '파주',
 '채',
 '돼',
 '시장',
 '와',
 '이상',
 '원',
 '게',
 '되며으로',
 '해',
 '도록',
 '에서',
 '씨',
 '니다',
 '카카오톡',
 '적인를',
 '항상',
 '고']

In [139]:
from konlpy.tag import Mecab
tokenizer = Mecab()


# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data


In [140]:
import time
start = time.time()  # 시작 시간 저장

text_data = preprocessing(df['news'])
print(text_data[:5])
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

['기사 섹션 분류 안내 기사 섹션 정보 해당 언론사 분류 를 따르 습니다 언론사 개별 기사 를 개 섹션 으로 중복 분류 습니다 닫 기', '제대 총선 실시간 개표 현황 결과 보 기 총선 여당 다시 번 승리 를 거두 면서 문재인 정부 경제 정책 힘 더 실리 됐 다는 평가 나온다 친 노동 큰 정부 복지 국가 내세우 정부 여당 정책 기조 더 강화 될 전망 경제 전문가 총선 이후 국회 집권 여당 역할 그 어느 때 보다 중요 다고 입 모았 올해 한국 경제 신종 코로나 바이러스 감염증 코로나 으로 극심 침체 를 겪 것 으로 예상 되 기 올해 한국 경제 년 외환 위기 이후 년 만 역 성장 것 으로 관측 되 국제통화기금 지난 일 한국 올해 경제 성장 률 전망 치 를 로 대폭 낮췄 코로나 로 인해 촉발 된 글로벌 경제 위기 를 한국 도 피해 기 힘들 것 란 이유 전문가 정부 여당 코로나 위기 극복 위해 적극 적 인 대책 수립 해야 지만 막대 재정 투입 되 만큼 우선 순위 를 정할 필요 다고 조언 안동현 서울대 경제 학부 교수 소비 진작 차원 긴급 재난 지원금 코로나 지원금 반대 경제학자 없 것 라면서 도 지급 대상 확대 기 보다 선별 통해 집중 적 으로 주 맞 다고 강조 도 재정 지원 적시 한시 적 선별 적 으로 제공 필요 다고 코로나 로 한계 상황 다다른 기업 대한 지원 책 시급 다는 주문 도 나왔 이인호 서울대 경제 학부 교수 한국 경제학 회장 기업 소상 공인 보다 안정 적 인 일자리 를 창출 다며 국회 기업 무너지 지 않 포괄 적 지원 나서 야 한다고 말 안동현 교수 주 시간 근로 제 를 유예 최저임금 제도 를 개선 해야 한다고 주장 외출 자제 영향 으로 매출 감소 대형 마트 휴일 영업 제한 철폐 소비 활성 화 될 해야 한다는 점 도 전문가 공통 된 의견 안 교수 기업 힘들 어 지 면 고용 불 안정 해진다며 국회 원점 경제 관련 법안 다시 들여다볼 필요 다고 말 코로나 완전히 종식 된 뒤 상황 대비 것 도 대 국회 과제 로 꼽힌다 전문가 내년 이후 재정 건전 성 문제 

# 3. 머신러닝 모델

In [101]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics



In [102]:
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

In [103]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 2997
테스트용 뉴스 기사의 개수 :  999
훈련용 레이블의 개수 :  2997
테스트용 레이블의 개수 :  999


In [104]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [105]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [106]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.94      0.75      0.83       237
          사회       0.79      0.92      0.85       423
       생활/문화       0.82      0.76      0.79       339

    accuracy                           0.83       999
   macro avg       0.85      0.81      0.82       999
weighted avg       0.83      0.83      0.82       999



### 다른 날짜 데이터 추가

In [110]:
import pandas as pd
import glob
import os

input_file =os.getenv("HOME") + "/aiffel/news_crawler" # csv파일들이 있는 디렉토리 위치
output_file = os.getenv("HOME") + "/aiffel/news_crawler/merge.csv"
allFile_list = glob.glob(os.path.join(input_file, 'news_*')) # glob함수로 sales_로 시작하는 파일들을 모은다
print(allFile_list)
allData = [] # 읽어 들인 csv파일 내용을 저장할 빈 리스트를 하나 만든다
for file in allFile_list:
    df = pd.read_csv(file, engine= 'python') # for구문으로 csv파일들을 읽어 들인다
    allData.append(df) # 빈 리스트에 읽어 들인 내용을 추가한다

dataCombine = pd.concat(allData, axis=0, ignore_index=True) # concat함수를 이용해서 리스트의 내용을 병합
# axis=0은 수직으로 병합함. axis=1은 수평. ignore_index=True는 인데스 값이 기존 순서를 무시하고 순서대로 정렬되도록 한다.
dataCombine.to_csv(output_file, index=False) # to_csv함수로 저장한다. 인데스를 빼려면 False로 설정

['/home/aiffel/aiffel/news_crawler/news_data2.csv', '/home/aiffel/aiffel/news_crawler/news_data.csv']


In [111]:
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/merge.csv"
df = pd.read_table(csv_path, sep=',')
df.head()

,news,code
0,기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...,경제
1,▶제21대 총선 실시간 개표 현황 및 결과 보기\n\n총선에서 여당이 다시 한 번 ...,경제
2,[뉴욕=AP/뉴시스]지난 10일 뉴욕 증권거래소 건물에 미국 국기가 게양되어 있다....,경제
3,"부산지역 주유소에서 판매하는 기름값이 휘발유는 평균 1200원대, 경유는 1000원...",경제
4,담배업계가 소비자의 취향을 저격한 다양한 담배 신제품들을 잇달아 선보이고 있다.이전...,경제


In [112]:
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       기사 섹션 분류 안내기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다 언론사...
1       제대 총선 실시간 개표 현황 및 결과 보기총선에서 여당이 다시 한 번 승리를 거두면...
2       뉴욕뉴시스지난 일 뉴욕 증권거래소 건물에 미국 국기가 게양되어 있다 신종 코로나바이...
3       부산지역 주유소에서 판매하는 기름값이 휘발유는 평균 원대 경유는 원대에 돌입했다일 ...
4       담배업계가 소비자의 취향을 저격한 다양한 담배 신제품들을 잇달아 선보이고 있다이전까...
                              ...                        
8822    신종 코로나바이러스 감염증코로나 사태 이후 가정의 달 월에도 언택트비대면 신풍속도가...
8823    는 소비자로부터 월 이용료 만만원을 받고 초고속 인터넷을 제공한다 그런 브로드밴드가...
8824    머리를 긁고 있는 오랑우탄 몸을 긁는 행동을 따라 하는 것은 부정적 감정과 관련이 ...
8825    가 오는 일 정식 출시하는 스마트폰 벨벳이 사실상 공짜폰이 될 전망이다 단말기 가격...
8826    이미지제공게티이미지뱅크 이미지제공게티이미지뱅크  전자신문  전자신문인터넷 무단전재 ...
Name: news, Length: 8827, dtype: object

In [113]:
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  6127


In [114]:
from konlpy.tag import Mecab
tokenizer = Mecab()


# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data


In [115]:
text_data = preprocessing(df['news'])
print(text_data[0])

기사 섹션 분류 안내 기사 섹션 정보 해당 언론사 분류 를 따르 고 습니다 언론사 개별 기사 를 개 이상 섹션 으로 중복 분류 습니다 닫 기


In [128]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'] ,random_state = 0)

In [118]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 4595
테스트용 뉴스 기사의 개수 :  1532
훈련용 레이블의 개수 :  4595
테스트용 레이블의 개수 :  1532


In [124]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [125]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [126]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.85      0.71      0.77       291
          경제       0.91      0.42      0.57       219
          사회       0.71      0.93      0.81       536
       생활/문화       0.78      0.78      0.78       486

    accuracy                           0.77      1532
   macro avg       0.81      0.71      0.73      1532
weighted avg       0.79      0.77      0.76      1532



## 생각보다 정확도가 높지 않았다.  그래서 다양한 모델로 실험해 보았다. RandomForest 와 SGDClassifier 을 써보았다

In [130]:
from sklearn.ensemble import RandomForestClassifier
#랜덤트리로 해보자!
                                                   
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = RandomForestClassifier().fit(X_train_tfidf, y_train)

In [131]:
y_pred= clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.78      0.81      0.79       271
          경제       0.83      0.60      0.70       225
          사회       0.82      0.90      0.86       559
       생활/문화       0.79      0.78      0.78       477

    accuracy                           0.80      1532
   macro avg       0.80      0.77      0.78      1532
weighted avg       0.80      0.80      0.80      1532



# 랜덤포레스트의 정확도도 80% 내외로 나왔다.

In [133]:
from sklearn.linear_model import SGDClassifier
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = SGDClassifier().fit(X_train_tfidf, y_train)

In [134]:
y_pred= clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.83      0.86      0.84       271
          경제       0.81      0.76      0.78       225
          사회       0.88      0.87      0.88       559
       생활/문화       0.79      0.82      0.80       477

    accuracy                           0.83      1532
   macro avg       0.83      0.82      0.83      1532
weighted avg       0.83      0.83      0.83      1532



# SGDClassifier을 사용했을 때 비로소 f1-score 83가 나왔다!